In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import datetime

import keras
from keras.models import Sequential
from keras.layers import Dense,InputLayer, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score


import numpy as np
from scikeras.wrappers import KerasClassifier
import keras_tuner
from tensorflow.keras import layers

2024-03-26 12:42:41.749812: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 12:42:42.991910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 12:42:42.991963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 12:42:42.996556: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 12:42:43.516397: I tensorflow/core/platform/cpu_feature_g

Using TensorFlow backend


# Global Model

In [2]:
# transformer model
n_timesteps, n_features = 240, 1
def transformer_encoder(inputs, head_size, num_heads, ff_dim,
                        dropout=0, attention_axes=None):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
      key_dim=head_size, num_heads=num_heads, dropout=dropout,
      attention_axes=attention_axes
      )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_transformer(head_size, 
                      num_heads,
                      ff_dim,
                      num_trans_blocks,
                      mlp_units, dropout, mlp_dropout, activation) -> tf.keras.Model:
    #n_timesteps, n_features, n_outputs = 240, 1, 1
    inputs = tf.keras.Input(shape=(n_timesteps, n_features))
    x = inputs 
    
    for _ in range(num_trans_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation=activation)(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(inputs, outputs)


def build_model(hp):
    head_size = hp.Choice("head_size",[32,64,128]) #embeding size for attention
    num_heads = hp.Choice("num_heads",[4,8,16])  #number of attention head
    ff_dim =  hp.Choice("ff_dim",[2,4,8,16,32])# hidden layer size in FNN insider transformer
    activation = hp.Choice("activation",["elu","relu","selu","tanh"])
    num_trans_blocks=  hp.Choice("num_trans_blocks",[2,4,8,16])
    #dropout = hp.Choice("drop_out",[0.3,0.5,0.7])
    #mlp_dropout= hp.Choice("mlp_dropout",[0.3,0.5,0.7])
    model = build_transformer(head_size, num_heads,ff_dim,num_trans_blocks,[256], 0.5, 0.5,activation)
    hp_lr = hp.Choice('learning_rate', values=[1e-6,1e-7,1e-8])
    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam',"adamax"])
    if hp_optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=hp_lr)
    elif hp_optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=hp_lr)
    elif hp_optimizer == 'adamax':
        optimizer = keras.optimizers.Adamax(learning_rate=hp_lr)
    else:
        raise ValueError("Invalid optimizer choice")

    model.compile(optimizer = hp_optimizer,loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
    return model

In [3]:
timesteps = 240
num_input =1
num_classes=1
label = list(range(timesteps)) + ['target'] + ['ticker'] + ['target_date'] + ['sector']

training_data = []
training_label = []
testing_data =[]
testing_label =[]

accuracy_results = []

for i in range(5):
    # read the data
    path = '/home/RDC/yeungwin/H:/yeungwin/SP500/data/'
    train = pd.read_csv(path+'Set_' + str(i) + '_Train.csv', index_col=0).dropna()
    test = pd.read_csv(path+'Set_' + str(i) + '_Test.csv', index_col=0).dropna()

    train.columns = label
    test.columns = label

    train_label = train.iloc[:, timesteps]
    train_data = train.iloc[:, :timesteps]
    test_label = test.iloc[:,timesteps]
    test_data = test.iloc[:, :timesteps]
    
    
     # reshape input
    #  data: (samples, timesteps, features)
    x_train = np.array(train_data).reshape((len(train_data), timesteps, num_input), order = 'F')
    x_test = np.array(test_data).reshape((len(test_data), timesteps, num_input), order = 'F')
    # label: (samples, target)
    y_train = np.array(train_label).reshape((len(train_label), num_classes))
    y_test = np.array(test_label).reshape((len(test_label), num_classes))
        
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    
    print("-------------------------------------------------------------------------------------------------------")
    print("Training the model for Training Set " + str(i) + " from " +
    datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
    print("-------------------------------------------------------------------------------------------------------")
        
    if i==0:
        tuner = keras_tuner.BayesianOptimization(build_model,
            objective='val_accuracy', #overwrite=True,
            #hyperband_iterations = 3,max_epochs =10,
            max_trials=30, directory='transformer', seed=10)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience =10, restore_best_weights=False)
        tuner.search(x_train,y_train, epochs =1000,validation_split=0.2, callbacks=[early_stop])

        # save the best model
        #hypermodel =build_model
        best_hp = tuner.get_best_hyperparameters()[0]
        best_model = build_model(best_hp)
        print(tuner.get_best_hyperparameters()[0].get_config()["values"])

        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10,restore_best_weights=False)
        result = best_model.fit(x_train,y_train, epochs=1000, batch_size=64,validation_split =0.2, verbose =1, callbacks=[early_stop])

        
    else:
        tuner = keras_tuner.BayesianOptimization(build_model,objective='val_accuracy', #overwrite=True,
            #hyperband_iterations = 3,max_epochs =10,
            max_trials=10, directory='transformer', seed=111)
        # save the best model
        best_hp = tuner.get_best_hyperparameters()[0]
        best_model = build_model(best_hp)
        load_path = '/home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight.h5'
        print('Model restore from ' + load_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=load_path,
                                                 save_weights_only=True,
                                                 verbose=1)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                    patience = 10, restore_best_weights=False)

        result = best_model.fit(
            x_train, 
            y_train, 
            epochs = 1000, 
            validation_split=0.2,
            verbose =1,
            callbacks=[cp_callback, early_stop]        
        ) 
        
    save_path = "/home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight.h5"
    best_model.save_weights(save_path)
    print("Model saved to " + save_path)
    print("Training end: " + datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
        
    ##make prediction
    pred_ff_test = best_model.predict(x_test)
    #pred = pred_ff_test.tolist()
    pred = pred_ff_test.reshape((1, len(pred_ff_test))).tolist()[0]
    output_data = pd.DataFrame({'y_prob': pred, 'y_true': test['target'], 'Ticker': test['ticker'],
                                    'Date': test['target_date'], 'Sector': test['sector'], })
    accuracy = accuracy_score(np.round(output_data['y_prob']), output_data['y_true'])
    print('Overall Accuracy for test set:'+ str(accuracy))
    
    
    output_data.to_csv('/home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/tf_pred/tf_prediction_period_'+str(i)+'.csv')
    print('Prediction for period ' + str(i) + ' successfully saved.')

(244381, 240, 1)
(244381, 1)
(119325, 240, 1)
(119325, 1)
-------------------------------------------------------------------------------------------------------
Training the model for Training Set 4 from 2024-02-09 14:28:33
-------------------------------------------------------------------------------------------------------
Reloading Tuner from transformer/untitled_project/tuner0.json
Model restore from /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
Epoch 1/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.7107 - accuracy: 0.5115
Epoch 1: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
6110/6110 [==============================] - 465s 76ms/step - loss: 0.7107 - accuracy: 0.5115 - val_loss: 0.6927 - val_accuracy: 0.5181
Epoch 2/1000
6109/6110 [============================>.] - ETA: 0s - loss: 0.6919 - accuracy: 0.5234
Epoch 2: saving model to /home/RDC/yeungwin/H:/yeun

Epoch 23/1000
6109/6110 [============================>.] - ETA: 0s - loss: 0.6378 - accuracy: 0.6292
Epoch 23: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
6110/6110 [==============================] - 478s 78ms/step - loss: 0.6378 - accuracy: 0.6293 - val_loss: 0.6618 - val_accuracy: 0.5908
Epoch 24/1000
6109/6110 [============================>.] - ETA: 0s - loss: 0.6379 - accuracy: 0.6301
Epoch 24: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
6110/6110 [==============================] - 475s 78ms/step - loss: 0.6378 - accuracy: 0.6301 - val_loss: 0.6610 - val_accuracy: 0.5909
Epoch 25/1000
6109/6110 [============================>.] - ETA: 0s - loss: 0.6358 - accuracy: 0.6330
Epoch 25: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
6110/6110 [==============================] - 477s 78ms/step - loss: 0.6358 - acc

6110/6110 [==============================] - 472s 77ms/step - loss: 0.5978 - accuracy: 0.6750 - val_loss: 0.6536 - val_accuracy: 0.6000
Epoch 70/1000
6109/6110 [============================>.] - ETA: 0s - loss: 0.5973 - accuracy: 0.6744
Epoch 70: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
6110/6110 [==============================] - 469s 77ms/step - loss: 0.5973 - accuracy: 0.6744 - val_loss: 0.6531 - val_accuracy: 0.6021
Model saved to /home/RDC/yeungwin/H:/yeungwin/SP500/6_Transfomer/transformer_model_weight/tf_weight3.h5
Training end: 2024-02-09 23:53:58
3729/3729 [==============================] - 95s 25ms/step
Overall Accuracy for test set:0.502828409805154
Prediction for period 4 successfully saved.


In [4]:
tuner = keras_tuner.BayesianOptimization(build_model,
    objective='val_accuracy',
    max_trials=10, directory='transformer3', seed=10)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience =3 )

# save the best model
print(tuner.get_best_hyperparameters()[0].get_config()["values"])

Reloading Tuner from transformer3/untitled_project/tuner0.json
{'head_size': 32, 'num_heads': 4, 'ff_dim': 8, 'activation': 'tanh', 'num_trans_blocks': 4, 'learning_rate': 1e-07, 'optimizer': 'rmsprop'}
